### Basic set up

In [1]:
pwd

'/home/pranshumaan'

In [2]:
cd /media/pranshumaan/TOSHIBA\ EXT

/media/pranshumaan/TOSHIBA EXT


In [3]:
cd Dev/

/media/pranshumaan/TOSHIBA EXT/Dev


In [4]:
ls

data/  Deep_Learning_Pytorch_Udemy_Tutorial/  Langchain/


In [5]:
cd Langchain/

/media/pranshumaan/TOSHIBA EXT/Dev/Langchain


In [6]:
ls

1_LangChain_Quickstart.ipynb  open_ai_api_key.txt


In [7]:
import os
import textwrap
 
import chromadb #vector database
import langchain
import openai
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma

In [8]:
%load_ext watermark
%watermark --iversions -v -m

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.12.0

Compiler    : GCC 11.3.0
OS          : Linux
Release     : 6.2.6-76060206-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 12
Architecture: 64bit

chromadb : 0.3.25
langchain: 0.0.139
openai   : 0.27.4



In [9]:
#Reduce output text width to fit in
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=100)))

In [10]:
print_response("Before diving into the Python interpreter, I pause here to remind you that this programming bootcamp is not meant to teach Python syntax (though you will lean that). The things you learn here are meant to help you understand, and ultimately do, computer programming more generally. Think of it this way: this bootcamp is meant to help you unleash the power of your computer on your biological problems. Python is just the language of instruction. That said, lets start talking about how Python works.Python s an interpreted language, which means that each line of code you write is translated, or interpreted, into a set of instructions that your machine can understand by the Python interpreter. This stands in contrast to compiled languages. For these languages (the dominant ones being Fortran, C, and C++), your entire code is translated into machine language before you ever run it. When you execute your program, it is already in machine language.So, whenever you want your Python code to run, you give it to the Python interpreter.There are many ways to launch the Python interpreter. One way is to type")

Before diving into the Python interpreter, I pause here to remind you that this programming bootcamp
is not meant to teach Python syntax (though you will lean that). The things you learn here are meant
to help you understand, and ultimately do, computer programming more generally. Think of it this
way: this bootcamp is meant to help you unleash the power of your computer on your biological
problems. Python is just the language of instruction. That said, lets start talking about how Python
works.Python s an interpreted language, which means that each line of code you write is translated,
or interpreted, into a set of instructions that your machine can understand by the Python
interpreter. This stands in contrast to compiled languages. For these languages (the dominant ones
being Fortran, C, and C++), your entire code is translated into machine language before you ever run
it. When you execute your program, it is already in machine language.So, whenever you want your
Python code to run, 

In [11]:
with open('open_ai_api_key.txt', 'r') as file:
    key = file.read().rstrip()

In [12]:
os.environ["OPENAI_API_KEY"] = key

In [13]:
# Createa a model object
model = OpenAI(temperature=0) #Default model is davinci003, 0 temperature means minimum randomness

In [14]:
print_response(model("You are the CEO of Awesome Spreadsheets Inc. Your spreadsheets are causing computers to catch fire. Draft a response for the Press in such a way that you are completely off the hook."))

  At Awesome Spreadsheets Inc., we take the safety of our customers and their computers very
seriously. We are aware of the reports of computers catching fire due to our spreadsheets and are
currently investigating the matter.  At this time, we have not been able to identify any specific
cause for the fires. We are working with our customers to ensure that their computers are safe and
that our spreadsheets are not causing any further issues.  We are committed to providing our
customers with the best possible experience and are taking all necessary steps to ensure that our
spreadsheets are safe and secure. We are also working with our customers to provide them with the
best possible support and assistance.  We apologize for any inconvenience this may have caused and
thank our customers for their patience and understanding.


### Q&A Over a Document

#### Reading from a URL

In [15]:
loader = WebBaseLoader(
    "https://blog.twitter.com/engineering/en_us/topics/open-source/2023/twitter-recommendation-algorithm"
)

In [16]:
loader

In [17]:
dir(loader)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_parser',
 '_fetch',
 '_fetch_with_rate_limit',
 '_scrape',
 'aload',
 'default_parser',
 'fetch_all',
 'load',
 'load_and_split',
 'requests_per_second',
 'scrape',
 'scrape_all',
 'session',
 'web_path',
 'web_paths']

In [18]:
documents = loader.load()
len(documents)

1

In [19]:
type(documents)

list

In [20]:
document = documents[0]
type(document)

langchain.schema.Document

In [21]:
document.__dict__.keys()

dict_keys(['page_content', 'metadata'])

In [22]:
document.metadata

{'source': 'https://blog.twitter.com/engineering/en_us/topics/open-source/2023/twitter-recommendation-algorithm'}

In [23]:
len(document.page_content)

12422

#### Vectorization and embedding generation

In [24]:
index = VectorstoreIndexCreator().from_loaders([loader])

#### Querying the document

In [25]:
query = "You're Spiderman. Explain the Twitter recommendation algorithm in 5 sentences using analogies from comic books."
print_response(index.query(query))

 The Twitter recommendation algorithm is like a web of interconnected services and jobs, like
Spiderman's web. The algorithm uses models and features to extract information from Tweets, users,
and engagement data, like Spiderman's spider-sense. The algorithm then generates candidate Tweets
and ranks them using a logistic regression model, like Spiderman's ability to sense danger. The
algorithm also uses heuristics and filters to make sure the Tweets are relevant, like Spiderman's
ability to dodge danger. Finally, the algorithm uses a service called Home Mixer to construct and
serve the For You timeline, like Spiderman's web-slinging ability to get from place to place.


#### Creating promps dynamically

In [26]:
template = """You're Dwight K. Schrute from the Office.
 
{context}
 
Answer with analogies from the Office to the question and the way Dwight speaks.
 
Question: {question}
Answer:"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])
print(
    prompt.format(
        context="Paper sales are declining 10% year over year.",
        question="How to sell paper?",
    )
)

You're Dwight K. Schrute from the Office.
 
Paper sales are declining 10% year over year.
 
Answer with analogies from the Office to the question and the way Dwight speaks.
 
Question: How to sell paper?
Answer:


In [27]:
pwd

'/media/pranshumaan/TOSHIBA EXT/Dev/Langchain'

In [28]:
!gdown 1eetuan04uj9-QKu_Vok2mbSK23G0H7yN
!gdown 1MVIhlCJS5RjVDy_s93Jb4vkHt6jAmgaa

/bin/bash: line 1: gdown: command not found
/bin/bash: line 1: gdown: command not found


### Loaders

In [30]:
import os
import textwrap

import chromadb
import langchain
import openai
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, UnstructuredPDFLoader, YoutubeLoader
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma


#### Text Loader

In [31]:
txt_loader = TextLoader("./the-need-to-read.txt", encoding="utf8")

In [32]:
txt_loader

In [41]:
# dir(txt_loader)

In [34]:
index = VectorstoreIndexCreator().from_loaders([txt_loader])

In [35]:
index

VectorStoreIndexWrapper(vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7fa9790e9660>)

In [39]:
# dir(index)

In [37]:
query = "Why someone in todays world would read? Answer in 3 sentences."
result = index.query_with_sources(query)
result

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


{'question': 'Why someone in todays world would read? Answer in 3 sentences.',
 'answer': ' Reading helps to develop critical thinking skills, encourages creativity, and allows for the discovery of new ideas. It also helps to develop writing skills, which are essential for problem solving. ',
 'sources': '[1] ./the-need-to-read.txt [2] ./the-need-to-read.txt'}

In [38]:
print_response(result["answer"])

 Reading helps to develop critical thinking skills, encourages creativity, and allows for the
discovery of new ideas. It also helps to develop writing skills, which are essential for problem
solving.


#### Youtube Loader

In [43]:
yt_loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=n2uY3-2Goek", add_video_info=True
)

In [46]:
yt_documents = yt_loader.load()
yt_documents

[Document(page_content="you get more out of reading one book that's great five times than out of reading five mediocre books if your behavior doesn't change as a result of reading a book and it means you learn nothing which means it was a waste of time and many people who read books are just wasting their time because their behavior doesn't change and so I consolidate once I find something that's good I plug everything I possibly can into it and suck the juice out of it so that I can change my behavior as a result which comes from the Frameworks and how I think about it so I read one thing that's very good many times rather than trying to brag about the fact that I read a book a week because I'm like what was the book last week not that good that it wasn't worth rereading", metadata={'source': 'n2uY3-2Goek', 'title': 'How to get the most out of reading', 'description': None, 'view_count': 31517, 'thumbnail_url': 'https://i.ytimg.com/vi/n2uY3-2Goek/hq720_2.jpg?sqp=-oaymwEoCIAKENAF8quKqQ

In [47]:
document = yt_documents[0]
document.page_content

"you get more out of reading one book that's great five times than out of reading five mediocre books if your behavior doesn't change as a result of reading a book and it means you learn nothing which means it was a waste of time and many people who read books are just wasting their time because their behavior doesn't change and so I consolidate once I find something that's good I plug everything I possibly can into it and suck the juice out of it so that I can change my behavior as a result which comes from the Frameworks and how I think about it so I read one thing that's very good many times rather than trying to brag about the fact that I read a book a week because I'm like what was the book last week not that good that it wasn't worth rereading"

#### PDF Loader

In [51]:
pdf_loader = UnstructuredPDFLoader("./Andrej_Karpathy_Resume.pdf")
pdf_pages = pdf_loader.load_and_split()

[nltk_data] Downloading package punkt to
[nltk_data]     /home/pranshumaan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/pranshumaan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [56]:
pdf_pages[0].page_content

'Andrej Karpathy\n\nandrej.karpathy@gmail.com http://cs.stanford.edu/~karpathy/\n\nEDUCATION\n\nStanford University (PhD), 2011 –\n\nComputer Science, studying Machine Learning and Computer Vision\n\nUniversity of British Columbia (Master’s degree), 2009 - 2011\n\nComputer Science graduate studies in Machine Learning, Vision, Motor Control - Average course grade: 94.4%\n\nUniversity of Toronto (Bachelor’s degree), 2005 - 2009\n\nDouble major in Computer Science and Physics, minor in Mathematics - Cumulative GPA: 3.74\n\nWORK EXPERIENCE\n\nGoogle Research (internship), June 2011 – September 2011\n\nDeveloped learning algorithms for video classification tasks - Worked on a large-scale learning framework for video analysis\n\nTeaching Assistant\n\n- -\n\n2011: Assisted with online offering of the Machine Learning class at Stanford 2011: Graduate Probabilistic Machine Learning class 2009-2010: Taught tutorial sections for a first year Discrete Mathematics class on four consecutive semester

In [61]:
type(pdf_pages[0])

langchain.schema.Document

In [63]:
# dir(pdf_pages[0])

#### Text Splitters
To overcome the token limits

In [66]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(pdf_pages)
len(texts)

2

In [67]:
texts[0]

Document(page_content='Andrej Karpathy\n\nandrej.karpathy@gmail.com http://cs.stanford.edu/~karpathy/\n\nEDUCATION\n\nStanford University (PhD), 2011 –\n\nComputer Science, studying Machine Learning and Computer Vision\n\nUniversity of British Columbia (Master’s degree), 2009 - 2011\n\nComputer Science graduate studies in Machine Learning, Vision, Motor Control - Average course grade: 94.4%\n\nUniversity of Toronto (Bachelor’s degree), 2005 - 2009\n\nDouble major in Computer Science and Physics, minor in Mathematics - Cumulative GPA: 3.74\n\nWORK EXPERIENCE\n\nGoogle Research (internship), June 2011 – September 2011\n\nDeveloped learning algorithms for video classification tasks - Worked on a large-scale learning framework for video analysis\n\nTeaching Assistant\n\n- -\n\n2011: Assisted with online offering of the Machine Learning class at Stanford 2011: Graduate Probabilistic Machine Learning class 2009-2010: Taught tutorial sections for a first year Discrete Mathematics class on fou

In [68]:
texts[1]

Document(page_content='COURSE WORK\n\n\n\nStanford: Machine Learning, Computer Vision, Convex Optimization, Probabilistic Graphical Models (I and II)\n\nUniversity of British Columbia: Machine Learning (I and II), Computer Vision (I\n\nand II)\n\nHACKING SKILLS\n\n\n\nPython, C++, MATLAB, Java, Objective C, Javascript/HTML/CSS, PHP, SQL\n\nINTERESTS Hobbies include Ping Pong, Ice skating, Scuba diving, PC strategy/fps games, Programming, and solving the Rubik’s cube in less than 20 seconds', metadata={'source': './Andrej_Karpathy_Resume.pdf'})

In [70]:
len(texts[0].page_content), len(texts[1].page_content)

(985, 455)

#### Embeddings

Downloading the sentence transformer model to use

In [72]:
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
hf_embeddings = HuggingFaceEmbeddings(model_name=MODEL_NAME)

In [73]:
text = texts[0].page_content
text

'Andrej Karpathy\n\nandrej.karpathy@gmail.com http://cs.stanford.edu/~karpathy/\n\nEDUCATION\n\nStanford University (PhD), 2011 –\n\nComputer Science, studying Machine Learning and Computer Vision\n\nUniversity of British Columbia (Master’s degree), 2009 - 2011\n\nComputer Science graduate studies in Machine Learning, Vision, Motor Control - Average course grade: 94.4%\n\nUniversity of Toronto (Bachelor’s degree), 2005 - 2009\n\nDouble major in Computer Science and Physics, minor in Mathematics - Cumulative GPA: 3.74\n\nWORK EXPERIENCE\n\nGoogle Research (internship), June 2011 – September 2011\n\nDeveloped learning algorithms for video classification tasks - Worked on a large-scale learning framework for video analysis\n\nTeaching Assistant\n\n- -\n\n2011: Assisted with online offering of the Machine Learning class at Stanford 2011: Graduate Probabilistic Machine Learning class 2009-2010: Taught tutorial sections for a first year Discrete Mathematics class on four consecutive semester

In [74]:
hf_embedding = hf_embeddings.embed_documents([text])
len(hf_embedding[0])

768

In [80]:
type(hf_embedding[0])

list

In [81]:
hf_embedding[0][:10]

[0.0018453436205163598,
 0.06233089044690132,
 -0.04533299431204796,
 -0.027133740484714508,
 0.010153498500585556,
 -0.013238425366580486,
 0.05982324481010437,
 -0.025168005377054214,
 -0.03709842264652252,
 -0.034538332372903824]

In [82]:
embeddings = OpenAIEmbeddings()
openai_embedding = embeddings.embed_documents([text])
len(openai_embedding[0])

1536

In [83]:
openai_embedding[0][:10]

[-0.0035485531035580205,
 0.013951345846037131,
 0.02112940819676652,
 -0.036430015031564046,
 0.012905669930165519,
 0.02726853926950074,
 -0.007184808385589652,
 0.018241991655223076,
 -0.03038532937819606,
 -0.020090476918771354]

#### Vectorstores

In [84]:
db = Chroma.from_documents(texts, embeddings)

In [85]:
db.similarity_search_with_score("What is the candidate work experience?", k=2)

[(Document(page_content='Andrej Karpathy\n\nandrej.karpathy@gmail.com http://cs.stanford.edu/~karpathy/\n\nEDUCATION\n\nStanford University (PhD), 2011 –\n\nComputer Science, studying Machine Learning and Computer Vision\n\nUniversity of British Columbia (Master’s degree), 2009 - 2011\n\nComputer Science graduate studies in Machine Learning, Vision, Motor Control - Average course grade: 94.4%\n\nUniversity of Toronto (Bachelor’s degree), 2005 - 2009\n\nDouble major in Computer Science and Physics, minor in Mathematics - Cumulative GPA: 3.74\n\nWORK EXPERIENCE\n\nGoogle Research (internship), June 2011 – September 2011\n\nDeveloped learning algorithms for video classification tasks - Worked on a large-scale learning framework for video analysis\n\nTeaching Assistant\n\n- -\n\n2011: Assisted with online offering of the Machine Learning class at Stanford 2011: Graduate Probabilistic Machine Learning class 2009-2010: Taught tutorial sections for a first year Discrete Mathematics class on f

#### Storing and Loading Embeddings

In [86]:
PERSIST_DIRECTORY = "db"

db = Chroma.from_documents(
    documents=texts, embedding=embeddings, persist_directory=PERSIST_DIRECTORY
)

In [87]:
db.similarity_search_with_score("What is the candidate work experience?", k=2)

[(Document(page_content='Andrej Karpathy\n\nandrej.karpathy@gmail.com http://cs.stanford.edu/~karpathy/\n\nEDUCATION\n\nStanford University (PhD), 2011 –\n\nComputer Science, studying Machine Learning and Computer Vision\n\nUniversity of British Columbia (Master’s degree), 2009 - 2011\n\nComputer Science graduate studies in Machine Learning, Vision, Motor Control - Average course grade: 94.4%\n\nUniversity of Toronto (Bachelor’s degree), 2005 - 2009\n\nDouble major in Computer Science and Physics, minor in Mathematics - Cumulative GPA: 3.74\n\nWORK EXPERIENCE\n\nGoogle Research (internship), June 2011 – September 2011\n\nDeveloped learning algorithms for video classification tasks - Worked on a large-scale learning framework for video analysis\n\nTeaching Assistant\n\n- -\n\n2011: Assisted with online offering of the Machine Learning class at Stanford 2011: Graduate Probabilistic Machine Learning class 2009-2010: Taught tutorial sections for a first year Discrete Mathematics class on f

In [88]:
db.persist()

In [89]:
# Loading data
vectordb = Chroma(persist_directory=PERSIST_DIRECTORY, embedding_function=embeddings)

In [90]:
vectordb.similarity_search_with_score("What is the candidate work experience?", k=2)

[(Document(page_content='Andrej Karpathy\n\nandrej.karpathy@gmail.com http://cs.stanford.edu/~karpathy/\n\nEDUCATION\n\nStanford University (PhD), 2011 –\n\nComputer Science, studying Machine Learning and Computer Vision\n\nUniversity of British Columbia (Master’s degree), 2009 - 2011\n\nComputer Science graduate studies in Machine Learning, Vision, Motor Control - Average course grade: 94.4%\n\nUniversity of Toronto (Bachelor’s degree), 2005 - 2009\n\nDouble major in Computer Science and Physics, minor in Mathematics - Cumulative GPA: 3.74\n\nWORK EXPERIENCE\n\nGoogle Research (internship), June 2011 – September 2011\n\nDeveloped learning algorithms for video classification tasks - Worked on a large-scale learning framework for video analysis\n\nTeaching Assistant\n\n- -\n\n2011: Assisted with online offering of the Machine Learning class at Stanford 2011: Graduate Probabilistic Machine Learning class 2009-2010: Taught tutorial sections for a first year Discrete Mathematics class on f

#### Creating a chain and using it on the saved data

In [91]:
chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0),
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
)

In [92]:
query = "What is the work experience of the candidate? Use no more than 2 sentences."
response = chain.run(query)

In [93]:
print_response(response)

The candidate has worked as an intern at Google Research, where he developed learning algorithms for
video classification tasks, and has also worked as a teaching assistant for various computer science
courses.


In [94]:
query = "Give a background summary of the candidate. Use no more than 3 sentences."
response = chain.run(query)
print_response(response)

Andrej Karpathy is a PhD student in Computer Science at Stanford University, where he is studying
Machine Learning and Computer Vision. He has a Master's degree in Computer Science from the
University of British Columbia and a Bachelor's degree in Computer Science and Physics from the
University of Toronto.


In [95]:
query = """
How likely is this candidate to be a top-tier DL researcher 2 years from now? 
Use 0-10 scale, where
0 - chance is nonexistent
10 - beyond reasonable doubt

You must choose a number and explain why
"""
response = chain.run(query)
print_response(response)

As an AI language model, I cannot predict the future, but based on the candidate's educational
background, work experience, and course work, it is evident that Andrej Karpathy has a strong
foundation in computer science, machine learning, and computer vision. He has also worked as a
research intern at Google, which is a significant achievement. Additionally, he has a good
understanding of programming languages such as Python, C++, and Java.   Considering all these
factors, I would rate his chances of becoming a top-tier DL researcher 2 years from now as 8 out of
10. However, it is essential to note that becoming a top-tier researcher requires more than just a
strong educational background and work experience. It also requires a passion for research,
dedication, and hard work.
